supplementary material for **Data management in life cycle assessment - a wastewater case study**, Julia Straub

**source code for automated LCI generation with WWTP database**

In [1]:
import pandas as pd
import json
import numpy as np
import openpyxl as opx

Load database with WWTP data

In [2]:
with open("database.json","r") as main:
    wwtp_data = json.load(main)

Create DataFrame that contains material, material type, quantity and unit for input and output site,<br>
materials obtained from ecoinvent 3.8 database by ecoinvent Association 2021,<br>
(except user generated materials: wastewater influent, sewage sludge wet, screenings, grit chamber trappings, wastewater effluent)

In [3]:
wastewater_treatment_df = pd.DataFrame(
{
    "mat_in": [
        "wastewater influent",
        "electricity, medium voltage",
        "natural gas, low pressure",
        "light fuel oil",
        "aluminium sulfate, without water, in 4.33% aluminium solution state",
        "iron(III) sulfate, without water, in 12.5% iron solution state",
        "iron(III) chloride, without water, in a 12% iron solution state",
        "iron(II) chloride",
        "sodium aluminate, powder",
        "polyaluminium chloride",
        " ",
        " ",
        " ",
        " "
    ],
    "type_in": [
        "Bad",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good",
        "Good"
    ],
    "quan_in": [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ],
    "unit_in":[
        "m³",
        "kWh",
        "m3",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        " ",
        " ",
        " ",
        " "
    ],
    "mat_out": [
        "BOD5, Biological Oxygen Demand [water/surface water]",
        "COD, Chemical Oxygen Demand [water/surface water]",
        "Ammonium, ion [water/surface water]",
        "Nitrate [water/surface water]",
        "Nitrite [water/surface water]",
        "Phosphorus [water/surface water]",
        "Methane, non-fossil [air/unspecified]",
        "Dinitrogen monoxide [air/unspecified]",
        "sewage sludge wet",
        "sewage sludge wet",
        "sewage sludge wet",
        "screenings",
        "grit chamber trappings",
        "wastewater effluent"
    ],
    "type_out": [
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Bad",
        "Neutral"
    ],
    "quan_out": [
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0,
        0.0
    ],
    "unit_out":[
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "kg",
        "t",
        "t",
        "t",
        "t",
        "t",
        "m³"
    ]
})

In [4]:
wastewater_treatment_df.iloc[10:14, 0:4] = np.nan

List with names of precipitants

In [5]:
precipitants = ['aluminium sulfate', 'iron(III) sulfate',
                'iron(III) chloride', 'iron(II) chloride',
                'sodium aluminate', 'polyaluminium chloride']

**Write values from WWTP database to DataFrame wastewater_treatment_df**

In [6]:
#Input
wastewater_treatment_df.at[0, 'quan_in'] = wwtp_data['influent']['wastewater_influent']
wastewater_treatment_df.at[1, 'quan_in'] = wwtp_data['energy']['electricity_from_grid']
wastewater_treatment_df.at[2, 'quan_in'] = wwtp_data['energy']['natural_gas_consumption']
wastewater_treatment_df.at[3, 'quan_in'] = wwtp_data['energy']['fuel_oil_consumption']

#precipitant
if wwtp_data['plant']['phosphorus_eliminiation'] == True:
    num = 0
    for i in range(6):
        if wwtp_data['plant']['phosphorus_precipitant'] == precipitants[i]:
            num = i + 4
            wastewater_treatment_df.at[num, 'quan_in'] = wwtp_data['plant']['precipitant_consumption']
#Output
#effluent [1]
wastewater_treatment_df.at[0, 'quan_out'] = wwtp_data['effluent']['BOD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3)
wastewater_treatment_df.at[1, 'quan_out'] = wwtp_data['effluent']['COD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3)
wastewater_treatment_df.at[2, 'quan_out'] = wwtp_data['effluent']['NH4-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3) * 1.29
wastewater_treatment_df.at[3, 'quan_out'] = wwtp_data['effluent']['NO3-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3) * 4.43
wastewater_treatment_df.at[4, 'quan_out'] = wwtp_data['effluent']['NO2-N'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3) * 3.28
wastewater_treatment_df.at[5, 'quan_out'] = wwtp_data['effluent']['Ptot'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3)

#GHG emissions[2]
#methane
wastewater_treatment_df.at[6, 'quan_out'] = wwtp_data['effluent']['BOD'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3) * 0.068

#nitrous oxide
n2o = (wwtp_data['influent']['totN'] * wwtp_data['influent']['wastewater_influent'] * 10**(-3) * 0.016 * 44/28) + \
(wwtp_data['effluent']['totN'] * \
(wwtp_data['influent']['wastewater_influent'] - wwtp_data['sludge']['sludge']['volume']) * 10**(-3) * 0.005 * 44/28)
wastewater_treatment_df.at[7, 'quan_out'] = n2o

#sewage sludge to disposal/dewatering
wastewater_treatment_df.at[8, 'quan_out'] = wwtp_data['sludge']['sludge']['volume'] * \
wwtp_data['disposal']['agriculture']
wastewater_treatment_df.at[9, 'quan_out'] = wwtp_data['sludge']['sludge']['volume'] * \
wwtp_data['disposal']['humification']
wastewater_treatment_df.at[10, 'quan_out'] = wwtp_data['sludge']['sludge']['volume'] * \
wwtp_data['disposal']['dewatering']

#screenings[3]
if wwtp_data['sludge']['screenings'] == None:
    wastewater_treatment_df.at[11, 'quan_out'] = wwtp_data['influent']['load'] * 1.4 * 10**(-3)
else:
    wastewater_treatment_df.at[11, 'quan_out'] = wwtp_data['sludge']['screenings']
    
#grit chamber trappings
wastewater_treatment_df.at[12, 'quan_out'] = wwtp_data['sludge']['grit_chamber_trappings']

#effluent
wastewater_treatment_df.at[13, 'quan_out'] = wwtp_data['influent']['wastewater_influent'] - \
wwtp_data['sludge']['sludge']['volume']

[1] The factors used to transform NH4-N, NO3-N and NO2-N into NH4, NO3, and NO2 are obtained from Landesamt fur Natur, Umwelt und Verbraucherschutz Nordrhein-Westfalen, https://www.abfall-nrw.de/abanda/help/1_umrechnung.htm

[2] The emissions of methane and nitrous oxide are calculated according to the guidelines of the IPCC, using the emission factors of the IPCC, 2019 Refinement to the 2006 IPCC Guidelines for National Greenhouse Gas Inventories: Volume 5: Waste,
Chapter 6: WASTEWATER TREATMENT AND DISCHARGE, https://www.ipcc-nggip.iges.or.jp/public/2019rf/vol5.html

[3] The factor to calculate the amount of screenings is obtained from Kuhn M., Gregor H. 2013. Rechengutanfall und -qualität. Korrespondenz Abwasser. 1/2013, Bd. 60., p. 30 - 36

**Transfer values of Dataframe to xlsx template**

In [7]:
wb = opx.load_workbook("templ.xlsx")
ws = wb.active
#input
for i in range(10):
    ws.cell(row = i + 4, column = 1, value = wastewater_treatment_df["mat_in"][i])
    ws.cell(row = i + 4, column = 4, value = wastewater_treatment_df["type_in"][i])
    ws.cell(row = i + 4, column = 6, value = wastewater_treatment_df["quan_in"][i])
    ws.cell(row = i + 4, column = 7, value = wastewater_treatment_df["unit_in"][i])
#output
for i in range(14):
    ws.cell(row = i + 4, column = 9, value = wastewater_treatment_df["mat_out"][i])
    ws.cell(row = i + 4, column = 12, value = wastewater_treatment_df["type_out"][i])
    ws.cell(row = i + 4, column = 14, value = wastewater_treatment_df["quan_out"][i])
    ws.cell(row = i + 4, column =15, value = wastewater_treatment_df["unit_out"][i])
wb.save("wastewater_treatment.xlsx")